In [ ]:
import os
import glob
import cv2
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [ ]:
file_path = "UCSF-PDGM-nifti"
file_list = glob.glob(os.path.join(file_path, "*"))
for i in file_list:
    img_list = glob.glob(os.path.join(i, "*"))
#     os.remove(img_list[3])
img_list

# Preprocessing

In [ ]:
def normalize(full_volume):
    """
    Z-Normalization of the whole subject
    """
    mu = full_volume.mean()
    std = np.std(full_volume)
    normalized = (full_volume - mu) / std
    return normalized

def standardize(normalized_data):
    """
    Standardize the normalized data into the 0-1 range
    """
    standardized_data = (normalized_data - normalized_data.min()) / (normalized_data.max() - normalized_data.min())
    return standardized_data

In [ ]:
def preprocessing(file_path):
    file_list = glob.glob(os.path.join(file_path, "*"))
    save_path = ("preprocessed")
    
    for counter, path in enumerate(tqdm(file_list)):
        patient_id = path[30:44]
        data_path = path + '/' + patient_id + '_T1.nii.gz'
        label_path = path + '/' + patient_id + '_tumor_segmentation.nii.gz'

        data = nib.load(data_path)
        label = nib.load(label_path)
        mri = data.get_fdata()
        mask = label.get_fdata().round()
        
        normalized_mri = normalize(mri)
        standardized_mri = standardize(normalized_mri)
        
        if counter < 480:
            current_path = os.path.join(save_path, "train", patient_id)
        else:
            current_path = os.path.join(save_path, "test", patient_id)
        
        # Loop over the slices in the full volume and store the images and labels in the data/mask directory
        for i in range(standardized_mri.shape[-1]):
            slice_mri = standardized_mri[:,:,i]
            slice_mask = mask[:,:,i]
            
            slice_mri_path = os.path.join(current_path, "data")
            slice_mask_path = os.path.join(current_path, "mask")
            if not os.path.exists(slice_mri_path):
                os.makedirs(slice_mri_path)
            if not os.path.exists(slice_mask_path):
                os.makedirs(slice_mask_path)
                
            np.save(os.path.join(slice_mri_path, str(i)), slice_mri)
            np.save(os.path.join(slice_mask_path, str(i)), slice_mask)

In [ ]:
file_path = "UCSF-PDGM-nifti/"
preprocessing(file_path)

# Validate proprocessed data

In [ ]:
path = "preprocessed/train/UCSF-PDGM-0004"
file = "115.npy"
slice_mri = np.load(path + "/data/" + file)
slice_mask = np.load(path + "/mask/" + file)

In [ ]:
slice_mask

In [ ]:
plt.figure()
plt.imshow(slice_mri, cmap="bone")
mask_ = np.ma.masked_where(slice_mask==0, slice_mask)
plt.imshow(mask_, cmap="autumn")